In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 - (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test

2022-08-08 13:50:34.083908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-08 13:50:34.083955: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Single Point on Sphere

n = 50000
rand_n = 12345
theta, phi = 1, 1
thetas, phis = theta * np.ones(n), phi * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

model = tf.keras.models.load_model('3dmodels/discrete_model_mth40_mph40')
# model = tf.keras.models.load_model('3dmodels/discrete_model_mth50_mph50')
# model = tf.keras.models.load_model('3dmodels/discrete_model_th3_ph3')

2022-08-08 13:50:39.603179: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-08 13:50:39.603231: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-08 13:50:39.603275: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-08-08 13:50:39.603667: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
for l in model.layers:
    l.trainable=False

inputs_hold = tf.keras.Input(shape=(1,))
initial_weights = np.ones(2)
# print(initial_weights)
simple_model = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
model3 = Model(inputs = inputs_hold, outputs = simple_model)

inputs = tf.keras.Input(shape=(2,))
inputs2 = tf.keras.layers.concatenate([inputs, model3(tf.ones_like(inputs)[:,0:1])])
hidden_layer_1 = model(inputs2)

model2 = Model(inputs = inputs, outputs = hidden_layer_1)
loss_fn = tf.keras.losses.BinaryCrossentropy()
model2.compile(loss=loss_fn, optimizer='Adam', )
# initialize model 2 weights to be the right answer (or near it, test how sensitive)

model2.summary()
print(model2.trainable_weights)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like (TFOpLambda)       (None, 2)            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1)            0           tf.ones_like[0][0]               
__________________________________________________________________________________________________
model (Functional)              (None, 2)            2           tf.__operators__.getitem[0][0]   
____________________________________________________________________________________________

In [ ]:
x_train_test = np.copy(x_train)
x_train_test[:, 2] = model2.trainable_weights[:][0][0][0] * (x_train[:, 2] - 2)
x_train_test[:, 3] = model2.trainable_weights[:][0][0][1] * (x_train[:, 3] - 2)
print(x_train_test[0])
model(x_train_test)

In [ ]:
coordinates = x_train[:, 0:2]
model2(coordinates)

In [4]:
x_train_mod = x_train
x_train_mod[:,2] = x_train[:, 2] - 1

In [5]:
names = []
names.append('discrete_model_th5_ph5')
names.append('discrete_model_th5_ph3')
names.append('discrete_model_th4_ph3')
names.append('discrete_model_th3_ph3')
names.append('discrete_model_th2_ph3')
names.append('discrete_model_th2_ph2')
names.append('discrete_model_th0_ph0')
names.append('discrete_model_mth15_mph15')

l = len(names)

models = []
predictions = []
for i in range(l):
    models.append(tf.keras.models.load_model('3dmodels/' + names[i]))
    xr = models[i](x_train).numpy().transpose()[0]
    loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
    print(names[i], 'has loss', loss)
    
# models = []
# predictions = []
# for i in range(l):
#     models.append(tf.keras.models.load_model('3dmodels/' + names[i]))
#     xr = models[i](x_train_mod).numpy().transpose()[0]
#     loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
#     print(names[i], 'has loss', loss)

discrete_model_th5_ph5 has loss 0.9759512744592924
discrete_model_th5_ph3 has loss 0.9746098276199208
discrete_model_th4_ph3 has loss 0.7359622403498043
discrete_model_th3_ph3 has loss 0.7679961990433425
discrete_model_th2_ph3 has loss 0.8718129085686916
discrete_model_th2_ph2 has loss 0.8254249363208351
discrete_model_th0_ph0 has loss 0.6929170613773167
discrete_model_mth15_mph15 has loss 0.6291996984028934


In [6]:
coordinates = x_train[:, 0:2]
# real_model = tf.keras.models.load_model('3dmodels/discrete_model_th3_ph3')
# x_r = real_model(x_train).numpy().transpose()[0]

for i in range(20):
    model2.fit(coordinates, y_train, epochs=1,batch_size=100)
    print(i,"Fitted result: ", model2.trainable_weights[:][0][0])
    x = model2(coordinates).numpy().transpose()[0]
    print(np.sum(-y_train * np.log(x) - (1-y_train) * np.log(1-x)) / x.shape[0])
    

2022-08-08 13:51:03.954538: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


800/800 [==============================] - 2s 2ms/step - loss: 0.6202
0 Fitted result:  tf.Tensor([1.1083084 1.0116988], shape=(2,), dtype=float32)
0.6196190572647301
800/800 [==============================] - 1s 2ms/step - loss: 0.6202
1 Fitted result:  tf.Tensor([1.1579126 1.018555 ], shape=(2,), dtype=float32)
0.6194100608199182
800/800 [==============================] - 1s 2ms/step - loss: 0.6205
2 Fitted result:  tf.Tensor([1.1853241 1.0079223], shape=(2,), dtype=float32)
0.6193857905057896
800/800 [==============================] - 1s 2ms/step - loss: 0.6201
3 Fitted result:  tf.Tensor([1.1884997 1.0149211], shape=(2,), dtype=float32)
0.6193449863747759
800/800 [==============================] - 1s 2ms/step - loss: 0.6199
4 Fitted result:  tf.Tensor([1.1946673 1.0140084], shape=(2,), dtype=float32)
0.6193344373527266
800/800 [==============================] - 1s 2ms/step - loss: 0.6202
5 Fitted result:  tf.Tensor([1.2277317 1.0073233], shape=(2,), dtype=float32)
0.619294669963191

In [ ]:
xr = model(x_train).numpy().transpose()[0]
y = model(x_train)
loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
print(loss)

In [ ]:
x_train_mod = np.copy(x_train)
x_train_mod[:, 2] = x_train[:, 2] - .1
x_train_mod[:, 3] = x_train[:, 3] - .46
xr_mod = model(x_train_mod).numpy().transpose()[0]
y_mod = model(x_train_mod)
loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
print(loss_mod)